In [8]:
import sys

## Analysis

The probability of Team A beating Team B at a stage $i$ to advance to stage $i+1$ is the following:

$$
P(A \text{ beat } B \text{ at stage } i) = P(A \text{ reaching stage }i) P(B \text{ reaching stage }i) P(A \text{ beat } B)
$$

So we need to calculate the probability of each team reaching every stage, i.e. BO8, quarter-final, semi-final, and final.

$$
P(A \text{ reaching stage } i + 1) = \sum_B P(A \text{ beat } B \text { at stage } i ) 
$$

The teams from the same bracket won't be meeting each other in later stage. For example, Brazil and Nigeria won't meet in the final as they are both from the upper bracket. We need to find a convenient representation of the brackets so that these impossible matchups can be excluded with ease.

Since there are only 16 teams, we can try binaries and use 4-bit to represent the brackets.

Brazil is the first team on the list, and thus can be represented as $0000$.

Argentina is the 7th team on the list, and thus can be represented as $(6)_2 = 0110$

BO8 matches are straightforward as the matchups are already decided.

For quarter-finals, we can check if the second lowest bit is the same. If yes, then this matchup is impossible as they already played against each other in BO8 matches.

For semi-finals, we check the second highest bit is the same.

For finals, we check the highest bit.

This way we can quickly decide which matchups are impossible.

Now time for implementation.

In [9]:
def read_data(file=None):
    """
    For demonstration purposes, the input was stored in the file.
    """
    countries = []
    probs = []
    if file is not None:
        sys.stdin = file
    
    for idx, line in enumerate(sys.stdin):
        if idx < 16:
            # First 16 lines are countries
            countries.append(line.strip())
        else:
            # Last 16 lines are probabilities
            probs.append([float(x) / 100 for x in line.strip().split()])
            
    return countries, probs

In [10]:
def calc_prob(prior, likelihood, phase):
    """
    phase = 1, BO8
    phase = 2, quarter-final
    phase = 3, semi-final
    phase = 4, final
    
    return: Probability of each country reaching phase i.
    """
    # Number of groups in each phase
    k = 16 // (2 ** phase)
    grouping = [set([2 ** phase * j + i for i in range(16 // k)]) for j in range(k)]
    
    # Probability of a country reaching next phase
    next_phase_prob = [0. for _ in range(16)]
    for group in grouping:
        for team in group:
            # A team cannot play against itself
            opponent = group - {team}
            
            for oppo in opponent:
                # Check if the team came from the same bracket as this opponent in the previous phase
                home_prev_phase_group = (team >> (phase - 1)) & 1  # Fetching the corresponding bit that represents the phase
                away_prev_phase_group = (oppo >> (phase - 1)) & 1
                if home_prev_phase_group != away_prev_phase_group:
                    # Can only play against each other if they came from different brackets in the previous phase
                    next_phase_prob[team] += prior[team] * prior[oppo] * likelihood[team][oppo]
    return next_phase_prob

In [11]:
def solve(country, probs):
    initial_prior = [1. for _ in range(16)]
    for i in range(1, 5):
        # Iterate through each phase
        next_phase_prior = calc_prob(initial_prior, probs, i)
        
        # Update prior to the current phase
        initial_prior = next_phase_prior
    
    for idx, c in enumerate(country):
        print("{:<10} p={:.2%}".format(c, initial_prior[idx]))

Example test case:

In [12]:
countries, probabilities = read_data(open("input.txt"))

In [13]:
solve(countries, probabilities)

Brazil     p=8.54%
Chile      p=1.60%
Nigeria    p=8.06%
Denmark    p=2.79%
Holland    p=4.51%
Yugoslavia p=7.50%
Argentina  p=8.38%
England    p=1.56%
Italy      p=9.05%
Norway     p=3.23%
France     p=13.72%
Paraguay   p=3.09%
Germany    p=13.79%
Mexico     p=3.11%
Romania    p=5.53%
Croatia    p=5.53%


In [ ]:
"""
This section is for OJ submission.
"""

if __name__ == "__main__":
    countries, probabilities = read_data()
    solve(countries, probabilities)